In [1]:
import pandas as pd
import geocoder
import numpy as np
from sklearn.metrics import r2_score
from scipy.stats import normaltest
import matplotlib.pyplot as plt 
import statsmodels.api as sm
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from statsmodels.stats.diagnostic import het_white,het_breuschpagan

#  ТЕКСТОВЫЙ АНАЛИЗ НА РЫНКЕ НЕДВИЖИМОСТИ
## Проектная работа по предмету количественные методы в экономике 
### Выполнили: Гончаров Глеб, Егоров Владимир
### Основано на "TEXTUAL ANALYSIS IN REAL ESTATE"
by ADAM NOWAK AND PATRICK SMITH

### Работе рассматриваются три модели
<center>1. Базовая</center>
$$ p_it=(z_it,a_1 )+(v_i,a_2 )+ε_i$$
<center>2. Модель с униграммами</center>
$$ p_it=(z_it,a_1 )+(v_i,a_2 )+ε_i$$
<center>3. Модель с биграммами</center>
$$ p_it=(z_it,a_1 )+(w_i,a_2 )+ε_i$$

In [2]:
def adj_r2_score(y_true, y_pred, n):
    return 1 - (1 - r2_score(y_true, y_pred)) * (y_true.shape[0] - 1 )/ ( y_true.shape[0] - n - 1)
def return_words(vectorizer, regresor, n):
    params = np.array(regresor.params[3:])
    good_ind = params.argsort()[-n:]
    bad_ind = params.argsort()[:n]
    good, bad = [],[]
    
    for i in good_ind:
        for key, value in vectorizer.vocabulary_.items():
            if value == i:
                good.append(key)
    for i in bad_ind:
        for key, value in vectorizer.vocabulary_.items():
            if value == i:
                bad.append(key)
    return good, bad

# I. Обработка первончальных данных, описательная статистика

### Вид первончальных данных

In [3]:
#Initail datased (parsed cian)
df = pd.read_csv('odi_cian.csv')
df.head()

,advAddress,metro,price,advName,description
0,"Московская область, Одинцово, мкр. 4, ул. Ново...",-,6 950 000,"4-комн. квартира, 65 м²","Продается 4-х комнатная квартира! г. Одинцово,..."
1,"Московская область, Одинцово, ул. Сколковская, 3А",-,4 485 000,"1-комн. квартира, 38 м²",Сколковский.ПЕРЕУСТУПКА от Собственника!!! Де...
2,"Московская область, Одинцово, ул. Гвардейская, 9","Молодежная, Крылатское, Кунцевская",5 450 000,"1-комн. квартира, 39 м²",Проедается новая квартира с качественной дизай...
3,"Московская область, Одинцово, Северная ул., 5к3","Славянский бульвар, Кунцевская, Молодежная",5 700 000,"1-комн. квартира, 32 м²",ЖК Одинбург! Квартира только после ремонта и у...
4,"Московская область, Одинцово, мкр. 7, 7-й микр...","Кунцевская, Славянский бульвар",2 402 000,"Студия, 25,4 м²","Продается квартира-студия площадью 25,4 кв.м н..."


### Из поля advName взяли количество комнат и квадратных метров, добавили дами-переменные для станций метро

In [4]:
#Added rooms and sq param
df['rooms'] = df['advName'].apply(lambda x: int(x[0]) if str.isdigit(x[0]) else 1)
df['sq'] = df['advName'].apply(lambda x: int(x.split(',')[-1].split(' ')[1]) 
                                if x.split(',')[-1].split(' ')[1] != 'м²' 
                                 else int(x.split(',')[-2].split(' ')[1]))
df.drop('advName',axis=1,inplace=True)
#Added dummies for subway stations
metr = []
for s in df['metro'].unique()[1:]:
    for j in s.split(','):
        if j.strip() not in metr:
            metr.append(j.strip())
for m in metr:
    df[m] = df['metro'].apply(lambda x : 1 if m in x else 0)
df.drop('metro',inplace=True,axis=1)

In [5]:
df.head()

,advAddress,price,description,rooms,sq,Молодежная,Крылатское,Кунцевская,Славянский бульвар,Киевская,...,Боровское шоссе,Солнцево,Деловой центр,Беговая,Белорусская,Говорово,Филевский парк,Тропарево,Саларьево,Пионерская
0,"Московская область, Одинцово, мкр. 4, ул. Ново...",6 950 000,"Продается 4-х комнатная квартира! г. Одинцово,...",4,65,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Московская область, Одинцово, ул. Сколковская, 3А",4 485 000,Сколковский.ПЕРЕУСТУПКА от Собственника!!! Де...,1,38,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Московская область, Одинцово, ул. Гвардейская, 9",5 450 000,Проедается новая квартира с качественной дизай...,1,39,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Московская область, Одинцово, Северная ул., 5к3",5 700 000,ЖК Одинбург! Квартира только после ремонта и у...,1,32,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,"Московская область, Одинцово, мкр. 7, 7-й микр...",2 402 000,"Продается квартира-студия площадью 25,4 кв.м н...",1,25,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


### Перевели адреса в координаты

In [ ]:
#Executes about 8 minutes
latlng = [] 
for s in df['advAddress']:
    latlng.append(geocoder.yandex(s).latlng)
df.drop('advAddress',axis=1,inplace=True)
latlng = np.array(latlng)
df['lat'] = latlng[:,0]
df['lon'] = latlng[:,1]
df.head()
col = ['price','rooms', 'sq','lat', 'lon','description', 'Молодежная', 'Крылатское',
       'Кунцевская', 'Славянский бульвар', 'Киевская', 'Парк Победы',
       'Юго-Западная', 'Новопеределкино', 'Фили', 'Рассказовка', 'Кутузовская',
       'Боровское шоссе', 'Солнцево', 'Деловой центр', 'Беговая',
       'Белорусская', 'Говорово', 'Филевский парк', 'Тропарево', 'Саларьево',
       'Пионерская']
df = df.reindex(columns=col)
df.to_csv('odi_latlon.csv',index=False)

Status code Unknown from https://geocode-maps.yandex.ru/1.x/: ERROR - HTTPSConnectionPool(host='geocode-maps.yandex.ru', port=443): Read timed out. (read timeout=5.0)


In [ ]:
df = pd.read_csv('odi_latlon.csv')
df.head()

### Мы поняли, что добавлять каждую станцию метро - это не оч хорошая идея, потому что они "идут" вместе. Объеденили станции в три группы: желтая и красная - по цвету веток, остальные - отдельная группа. Получили вид итогового датасета.

In [ ]:
#To add subways isn't really good idea, so we united them in groups
railway = ['Молодежная', "Крылатское", "Кунцевская", "Славянский бульвар", "Киевская","Парк Победы","Фили", "Кутузовская","Деловой центр", "Филевский парк", "Пионерская","Беговая", "Белорусская"]
red = ["Юго-Западная", "Тропарево","Саларьево"]
yellow =["Новопеределкино","Рассказовка", "Боровское шоссе", "Солнцево","Говорово"]
col = df['Молодежная'].copy()
for i in railway[1:]:
    col += df[i]
df['railway'] = col.apply(lambda x: 1 if x >= 1 else 0)
col = df['Юго-Западная'].copy()
for i in red[1:]:
    col += df[i]
df['red'] = col.apply(lambda x: 1 if x >= 1 else 0)
col = df['Новопеределкино'].copy()
for i in yellow[1:]:
    col += df[i]
df['yellow'] = col.apply(lambda x: 1 if x >= 1 else 0)
df.drop(railway,axis=1,inplace=True)
df.drop(red,axis=1,inplace=True)
df.drop(yellow,axis=1,inplace=True)
df['price'] = df['price'].apply(lambda x: x.replace(' ',''))
df['price'].astype('int64')
df['description'] = df['description'].apply(lambda x: str.lower(x))
df.head()
df.to_csv('odi_final.csv',index=False)

In [ ]:
df = pd.read_csv('odi_final.csv')
df.head()

### Корреляция между квадратными метрами и количеством комнта сильная - убираем количество комнат из модели. 

In [ ]:
df.corr()

In [ ]:
df.describe()

### Объявление с максимальной ценой

In [ ]:
df[df['price'] == df['price'].max()].T

## II. Model #1. Simple Linear Regression

In [ ]:
df.drop('rooms',axis=1,inplace=True)
X_train, X_test, Y_train, Y_test  = train_test_split(df.drop(['price'], axis=1),
                                                     np.log(df['price']), test_size=0.2, random_state=3)
lr = sm.OLS(Y_train,X_train.drop(['description'],axis=1)).fit()
print(lr.summary())

### R2 и Adjusted R2

In [ ]:
print('In sample r2:',adj_r2_score(Y_train,lr.predict(X_train.drop('description',axis=1)),X_train.shape[1] - 1), 
      'Out of sample r2:',adj_r2_score(Y_test,lr.predict(X_test.drop('description',axis=1)),X_train.shape[1]))

### Проверка предпосылок теоремы Гаусса-Маркова
1. Равенство матожидания 0

In [ ]:
eps = Y_train - lr.predict(X_train.drop('description',axis=1))
print('Residuals mean:', eps.mean())

2. Независимость остатков и переменных видна из графика

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(range(len(eps)),eps,s=3);

3. Согласно тесту условие гетероскедастичности не выполняется

In [ ]:
p_value = het_breuschpagan(eps,lr.model.exog)[-1]
print('P-value:', p_value)

4. Не понимаем, как проверить ковариации остатков

### Проверка нормальности распредления остатков
Из теста на нормальность, как и из графика видно, что остатки не распределны нормально

In [ ]:
plt.figure(figsize=(8,8))
plt.hist(eps)
print('P-value:', normaltest(eps)[-1], '\np-value почти ноль - отклоняем нулевую гипотезу, поэтому распредление остатков не является нормальным')

## Model #2. Linear Regression with Unigrams

In [ ]:
stop_words = stopwords.words('russian')
vectorizer_uni = CountVectorizer(min_df=120,stop_words=stop_words)
train_uni_descr = vectorizer_uni.fit_transform(X_train['description'])
test_uni_descr = vectorizer_uni.transform(X_test['description'])
train_uni_descr.shape

In [ ]:
X_train_unigram = np.hstack([X_train[['sq','lat','lon']], train_uni_descr.toarray()])
X_test_unigram = np.hstack([X_test[['sq','lat','lon']], test_uni_descr.toarray()])
X_train_unigram.shape

In [ ]:
X_test_unigram[0,:]

In [ ]:
lr_uni = sm.OLS(Y_train,X_train_unigram).fit()
print(lr_uni.summary())

### R2 и Adjusted R2

In [ ]:
print('In sample r2:',adj_r2_score(Y_train,lr_uni.predict(X_train_unigram),X_train.shape[1]), 
      'Out of sample r2:',adj_r2_score(Y_test,lr_uni.predict(X_test_unigram),X_train.shape[1]))

In [ ]:
good_uni, bad_uni = return_words(vectorizer_uni, lr_uni,20)
print('Good ones:\n', good_uni)
print('Bad ones:\n',bad_uni)

<div style='float:left'><img width=400 src="1.png" alt="HSE logo"></img></div> 
<div style='float:right'><img width=400 src="2.png" alt="HSE logo"></img></div> 



## Model #3. Linear Regression with Bigrams

In [ ]:
vectorizer_bi = CountVectorizer(min_df=76,stop_words=stop_words,ngram_range=(2,2))
train_bi_descr = vectorizer_bi.fit_transform(X_train['description'])
test_bi_descr = vectorizer_bi.transform(X_test['description'])
train_bi_descr.shape

In [ ]:
X_train_bigram = np.hstack([X_train[['sq','lat','lon']], train_bi_descr.toarray()])
X_test_bigram = np.hstack([X_test[['sq','lat','lon']], test_bi_descr.toarray()])
X_train_bigram.shape

In [ ]:
lr_bi = sm.OLS(Y_train,X_train_bigram).fit()
print(lr_bi.summary())

In [ ]:
print('In sample r2:',adj_r2_score(Y_train,lr_bi.predict(X_train_bigram),X_train.shape[1]), 
      'Out of sample r2:',adj_r2_score(Y_test,lr_bi.predict(X_test_bigram),X_train.shape[1]))

In [ ]:
good_bi, bad_bi = return_words(vectorizer_bi, lr_bi,20)
print('Good ones:\n', good_bi)
print('Bad ones:\n', bad_bi)

<div style='float:left'><img width=400 src="3.png" alt="HSE logo"></img></div> 
<div style='float:right'><img width=400 src="4.png" alt="HSE logo"></img></div> 
